# cr3 CTF 2024
##   pudge  | 317 pts

Task description:


![](./pUdge.png)

```
author: kibastian
```

Attachments:

In [ ]:
from sage.all import *
from Crypto.Util.number import bytes_to_long

FLAG = f'cr3{{{"_"*34}}}'.encode()
assert len(FLAG) == 39

flag = bytes_to_long(FLAG)

# p = random_prime(2^64-1, True, 2^60)
p = 14989068425250509519
pudge = 488021842552697485963550845809830956869265678611663884519824835144279897263760519070556811395650

# A = random_matrix(GF(p), 5,5)
A = matrix(GF(p), [
    [2316580867395718488, 48426743017495202, 10035688260900924039, 4231347989577694113, 10589292030513420982],
    [13855044643446484404, 5232761037656480890, 8292975196384415335, 6347855203040361884, 1695816437660153737],
    [13046242976127835996, 9475110516100642016, 4727306285316450578, 4641412090997841438, 12402034468342504110],
    [2367861295557041099, 7972370016088296064, 2345756074442706631, 5466959662134124819, 7703316732710858490],
    [3154951152478314029, 14631229995381692381, 8791489668576804292, 14058045966950055197, 6886936172143671821]
])
# A_1 = PUDGE?
A_1 = matrix(GF(p), [
    [10449681695550103501, 1032827731769907364, 8551846352768290089, 2528206462071885652, 4668517139358847432],
    [8129949521585158431, 8663150094657298270, 13611618663064384279, 11990726265981907640, 8736574751606166199],
    [13477489095760140976, 4129645872383074495, 802354481887101565, 5918883130626557767, 4293067273646165724],
    [3218110987866379647, 13155490303126428937, 10511509599742650284, 10905345492015529131, 8890727867148431908],
    [6267270344792504320, 1272127850159541188, 9155031196436204158, 13829185043890037978, 2887328975771996112]
])


def pUdge(s=1):
    a_1 = A
    if s == 1:
        return a_1
    a_2 = A_1
    if s == 2:
        return a_2
    s -= 2
    while True:
        a_n = a_2 * a_1
        a_1, a_2 = a_2, a_n
        if s == 1:
            return a_n
        s -= 1


assert pUdge(2 ** 322 - 228) == A ** (flag + pudge)

## Solution

Here we have a recursive sequence. 
First thought should be to check whether these matrices commute. They are. But it was not my first thought.

Let's take a look at first several elements of this sequence:

$a_3 = a * a_1$

$a_4 = a * a_1 * a_1 = a * a_1^2$

$a_5 = (a * a_1^2) * (a * a_1) = a^2 * a_1^3$

$a_6 = (a^2 * a_1^3) * (a * a_1^2) = a^3 * a_1^5$

Yeah, we can see our boi Fibonacci.

So $pUdge(m) = A^{fib_{m-2}} * A_1^{fib_{m-1}}$, keeping in mind that the seed is 0, 1

In [2]:
p = 14989068425250509519
pudge = 488021842552697485963550845809830956869265678611663884519824835144279897263760519070556811395650

A = Matrix(GF(p), [
    [2316580867395718488, 48426743017495202, 10035688260900924039, 4231347989577694113, 10589292030513420982],
    [13855044643446484404, 5232761037656480890, 8292975196384415335, 6347855203040361884, 1695816437660153737],
    [13046242976127835996, 9475110516100642016, 4727306285316450578, 4641412090997841438, 12402034468342504110],
    [2367861295557041099, 7972370016088296064, 2345756074442706631, 5466959662134124819, 7703316732710858490],
    [3154951152478314029, 14631229995381692381, 8791489668576804292, 14058045966950055197, 6886936172143671821]
])
# A_1 = PUDGE?
A_1 = Matrix(GF(p), [
    [10449681695550103501, 1032827731769907364, 8551846352768290089, 2528206462071885652, 4668517139358847432],
    [8129949521585158431, 8663150094657298270, 13611618663064384279, 11990726265981907640, 8736574751606166199],
    [13477489095760140976, 4129645872383074495, 802354481887101565, 5918883130626557767, 4293067273646165724],
    [3218110987866379647, 13155490303126428937, 10511509599742650284, 10905345492015529131, 8890727867148431908],
    [6267270344792504320, 1272127850159541188, 9155031196436204158, 13829185043890037978, 2887328975771996112]
])

A * A_1 == A_1 * A

True

Well, they commute. So they are probably in the same mutltiplicative subgroup.

Let's try to solve dlp...

In [4]:
G1 = GL(5, GF(p))
discrete_log(G1(A_1), G1(A))

3

Cool. So $pUdge(m) = A^{fib_{m - 2} + 3 * fib_{m-1}}$

In [5]:
order = G1(A).order()
order

756611975305664976948701508934220455553386636654110694421436693429248935533058528391714742723598

Sage factor was running for more than 5 seconds so I called for ma friend yafu. Factorization was done in less than a second:

```
P1 = 2
P2 = 41
P3 = 181
P3 = 281
P4 = 1021
P9 = 252802471
P16 = 7340386104432179
P16 = 7160174515257691
P25 = 2533264055405865190411771
P22 = 5278912143021689094931
```

In [6]:
orders = [
    2,
    41,
    181,
    281,
    1021,
    252802471,
    7160174515257691,
    7340386104432179,
    2533264055405865190411771,
    5278912143021689094931,
]
orders = sorted(orders)
assert prod(orders) == order

We need this factorization to perform fast fibonacci calculation in prime order groups.

### Fibonacci basics.

Using some generating sequence theory or just checking out wiki we have:

$fib(n) = \frac{\phi^n - \psi^n}{\sqrt{5}}$, $\phi = \frac{1 + \sqrt(5)}{2}$, $\psi = -\phi^{-1}$

Very conviniently $\sqrt{5}$ exists modulo all the prime factors of the order:

In [7]:
assert all(GF(p)(5).is_square() for p in orders)

In [8]:
def pUdge_power(n, p):
    G = GF(p)
    sq = G(5).sqrt()
    phi = (1 + sq) / 2
    psi = (1 - sq) / 2
    
    n = (n - 2) % (p -1)
    res1 = (pow(phi, n) - pow(psi, n)) / sq
    n += 1
    
    res2 = (pow(phi, n) - pow(psi, n)) / sq

    alpha = res1 + 3 * res2
    return alpha

After all that we can CRT the results to obtain $pUdge power(n) \pmod{order}$

Also note that 2 is not invertible mod 2, but $m = 2^{322} - 228 = 1 \pmod{3}$ which sequence order $\pmod{2}$

In [10]:
m = 2**322 - 228
fibs = [1] + [int(pUdge_power(m, p)) for p in orders[1:]]
res = crt(fibs, orders) - pudge
print(int(res).to_bytes(50, 'big'))

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00cr3{1_luv_PUDGE_g0D_I_w1Sh_1_w45_puDG3}'
